### Initialize

In [2]:
from datetime import datetime
import json
import os
import sys

from ipywidgets import Dropdown
import pandas as pd
import requests

sys.path.append(os.path.join(sys.path[0]))
from tools import print_response, StopExecution, get_base_url, get_root_path
import esh_objects as esh

base_url = get_base_url()
root_path = get_root_path()
selected = sys.path[0].split(os.sep)[-2]

def execute_search_query(query):
    r = requests.post(f'{base_url}/v1/query/{selected}Example/latest', json=[query])
    if r.status_code == 200:
        res = r.json()[0]
        if '@odata.count' in res:
            print(f'Total count: {res["@odata.count"]}')
        if res and 'value' in res and res['value']:
            df = pd.DataFrame.from_dict(res['value'])[['firstName', 'lastName','age']]
            df.rename(columns={'firstName': 'First Name', 'lastName': 'Last Name', 'age':'Age'}, inplace=True)
            display(df)
        else:
            print('No results')
    else:
        print(f'Error during search. HTTP status code {r.status_code}')
        print(r.json())



### Fuzzy-search for first name "Alwx"
-> It is searched for the first name with 70 percent match

In [3]:
so = esh.EshObject(
    count=True,
    # top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items= [
            esh.ScopeComparison(values='Person'),
            esh.Comparison(
                property= esh.Property(property='firstName'),
                operator= esh.ComparisonOperator.Search,
                value= esh.StringValue(
                    value='Alwx',
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7)
                )
            )
        ]
    )
)

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))



ESH query statement: /$all?$top=10&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND firstName:Alwx~0.7'))
Total count: 1


,First Name,Last Name,Age
0,Alex,Kennedy,57


## Fuzzy-search with first name and lastname 
-> It is searchef for the first name and last name with a 70 percent match

In [3]:
so = esh.EshObject(
    count=True,
    # top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items= [
            esh.ScopeComparison(values='Person'),
            esh.Comparison(
                property= esh.Property(property='firstName'),
                operator= esh.ComparisonOperator.Search,
                value= esh.StringValue(
                    value='Alwx',
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7)
                )
            ),
            esh.Comparison(
                 property= esh.Property(property='lastName'),
                operator= esh.ComparisonOperator.Search,
                value= esh.StringValue(
                    value='Konnedy',
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7)
            )
            )
        ]
    )
)

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))



ESH query statement: /$all?$top=1&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND firstName:Alwx~0.7 AND lastName:Konnedy~0.7'))
Total count: 1


,First Name,Last Name,Age
0,Alex,Kennedy,57


## Fuzzy-search with place of birth and first name
-> It is searched for a birst with place of birth and first name with a 70 percent match

In [52]:
scope = esh.ScopeComparison({esh.Constants.values: ['Person']})
comparison = []
comparison.append(esh.Comparison({
    esh.Constants.property: esh.Property({esh.Constants.property: 'placeOfBirth'}),
    esh.Constants.operator: ':',
    esh.Constants.value: esh.Term(
        {esh.Constants.term: 'Sutggart',
         esh.Constants.searchOptions:
            { esh.Constants.fuzzinessThreshold:0.7} })
    }))
comparison.append(esh.Comparison({
    esh.Constants.property: esh.Property({esh.Constants.property: 'firstName'}),
    esh.Constants.operator: ':',
    esh.Constants.value: esh.Term(
        {esh.Constants.term: 'Hildo',
         esh.Constants.searchOptions:
            { esh.Constants.fuzzinessThreshold:0.7} })
    }))    
exp = esh.Expression({esh.Constants.items:[scope,comparison[0], comparison[1]]})
exp.operator = 'AND'
so = esh.IESSearchOptions({esh.Constants.searchQueryFilter: exp})
print(so.to_statement())
execute_search_query(so.to_dict())

/$all?$top=10&$apply=filter(Search.search(query='SCOPE:Person AND placeOfBirth:Sutggart~0.7 AND firstName:Hildo~0.7'))


,First Name,Last Name,Age
0,Hilda,Friedel,51


## Fuzzy-search with nationality, place of birth first name and last name
--> This fuzzy search includes multiple conditions with a 70 percent match

In [4]:
so = esh.EshObject(
    count=True,
    # top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items= [
            esh.ScopeComparison(values='Person'),
            esh.Comparison(
                property= esh.Property(property='firstName'),
                operator= esh.ComparisonOperator.Search,
                value= esh.StringValue(
                    value='Ivon',
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7)
                )
            ),
            esh.Comparison(
                 property= esh.Property(property='lastName'),
                operator= esh.ComparisonOperator.Search,
                value= esh.StringValue(
                    value='Kowalsko',
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7)
            )
            ),
            esh.Comparison(
                property= esh.Property(property='placeOfBirth'),
                operator= esh.ComparisonOperator.Search,
                value= esh.StringValue(
                    value='Warsow',
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7)
                )
            ),
            esh.Comparison(
                property= esh.Property(property='nationality'),
                operator= esh.ComparisonOperator.Search,
                value= esh.StringValue(
                    value='Uk',
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7)
                )
            )
        ]
    )
)

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))



ESH query statement: /$all?$top=10&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND firstName:Ivon~0.7 AND lastName:Kowalsko~0.7 AND placeOfBirth:Warsow~0.7 AND nationality:Uk~0.7'))
Total count: 1


,First Name,Last Name,Age
0,Ivan,Kowalski,65


## Search for persons who are approx. 30 years old
--> It is searched for persons who are approx. 30 years old. Therefore the field "age" is set as property and the term "30" is entered. Due to the age 30 has just to digits the fuzzinesThreshold is set to 0.5 precision (50% match).

In [5]:
so = esh.EshObject(
    count=True,
    top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Person'),
            esh.Comparison(
                property=esh.Property(property='age'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(
                    value=30,
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.5)
                )
            )
        ]
    )
)

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))


ESH query statement: /$all?$top=1&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND age:30~0.5'))
Total count: 2


,First Name,Last Name,Age
0,Joseph,Kennedy,36
